In [87]:
import csv

In [88]:
import random

In [89]:
import math

In [109]:
import copy

In [108]:
import pandas as pd 

In [91]:
def loadCsv(filename):
	lines = csv.reader(open(r'diabetes.csv'))
	dataset = list(lines)
	classes=copy.copy(dataset[0:1])
	dataset=copy.copy(dataset[1:])
	
	for i in range(len(dataset)):
		dataset[i] = [float(x) for x in dataset[i]]
	return dataset

In [92]:
def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]
 

In [93]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated


In [94]:
def mean(numbers):
	return sum(numbers)/float(len(numbers))
 #Desvio padrão 
def stdev(numbers): 
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

In [95]:
def stdev(numbers):
 
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [96]:
def summarize(dataset):


    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]

    del summaries[-1]
    return summaries

In [115]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    
    summaries = {}
    for classValue, instances in separated.items():
        

        summaries[classValue] = summarize(instances)
    
#     print("\nTamanho",len(summaries) ,summaries)
    print(pd.DataFrame.from_dict(summaries))
    return summaries

In [98]:
def calculateProbability(x, mean, stdev):
    print("Valor X",x,"Media",mean,"Desvio",stdev,"\n")
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [99]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
#         print("Tamanho", len(classSummaries),classSummaries,'\n')
        
        print(inputVector, "Tamanho", len(inputVector),"\n")
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    
    return  probabilities


In [124]:
def predict(summaries, inputVector):
    calculateClassProbabilities(summaries, inputVector)
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
            
    return bestLabel

In [101]:
def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
#	print(predictions,"\n")
	return predictions

In [102]:
def getAccuracy(testSet, predictions):
	correct = 0
	valorA=0
	valorB=0
	status=-1
	situacao=-1

	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct+=1
			if testSet[i][-1] ==1.0:
				valorA+=1
			elif testSet[i][-1] == 0.0:
				valorB+=1
	print(correct,"A",valorA,"B",valorB)
	if valorA>valorB:
		situacao=(valorA/float(correct)) * 100.0
		status=1
	else:
		situacao=(valorB/float(correct)) * 100.0
		status=0
		
	#Buscando qual a classe teve o maior acerto, acredito que estou no caminho certo, agora e fazer o teste final 
	accuracy=(correct/float(len(testSet))) * 100.0

	return accuracy,status,situacao

In [122]:
def main():
	filename = 'diabetes.csv'
	splitRatio = 0.67
	status=-1
	situacao=-1
	dataset = loadCsv(filename)
	trainingSet, testSet = splitDataset(dataset, splitRatio)
	print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
	# prepare model
	summaries = summarizeByClass(trainingSet)
	# test model
	predictions = getPredictions(summaries, testSet)
	accuracy,status,situacao = getAccuracy(testSet, predictions)
	print('Accuracy: {0}%'.format(accuracy))
	if status==0:
		result="Não teve diabetes" 
	else:
		result="Teve diabetes"
	print("S",result,"{0}%".format(situacao))

In [125]:
main()

Split 768 rows into train=514 and test=254 rows
                                        0.0  \
0  (3.3488372093023258, 2.9604484801702973)   
1   (109.89244186046511, 26.72476168863615)   
2    (68.63372093023256, 17.80407206322981)   
3  (19.584302325581394, 14.905685536714216)   
4              (70.875, 102.61851038418648)   
5   (30.583720930232595, 7.178764610500314)   
6  (0.4321598837209308, 0.3027708011617666)   
7  (31.337209302325583, 11.800440239963061)   

                                         1.0  
0    (4.847058823529411, 3.7637507490084037)  
1   (141.58823529411765, 30.958569037786383)  
2    (70.87058823529412, 22.047977934787543)  
3   (21.323529411764707, 17.660038445137808)  
4   (108.95882352941176, 154.94375218705324)  
5    (34.627647058823534, 7.411162242985588)  
6  (0.5230058823529418, 0.33234979250870283)  
7    (37.33529411764706, 10.648190177443055)  
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0, 1.0] Tamanho 9 

Valor X 8.0 Media 3.3488372093023258 Des

Valor X 0.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 118.0 Media 109.89244186046511 Desvio 26.72476168863615 

Valor X 84.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 47.0 Media 19.584302325581394 Desvio 14.905685536714216 

Valor X 230.0 Media 70.875 Desvio 102.61851038418648 

Valor X 45.8 Media 30.583720930232595 Desvio 7.178764610500314 

Valor X 0.551 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 31.0 Media 31.337209302325583 Desvio 11.800440239963061 

[0.0, 118.0, 84.0, 47.0, 230.0, 45.8, 0.551, 31.0, 1.0] Tamanho 9 

Valor X 0.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 118.0 Media 141.58823529411765 Desvio 30.958569037786383 

Valor X 84.0 Media 70.87058823529412 Desvio 22.047977934787543 

Valor X 47.0 Media 21.323529411764707 Desvio 17.660038445137808 

Valor X 230.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 45.8 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.551 Media 0.52

Valor X 0.388 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 50.0 Media 31.337209302325583 Desvio 11.800440239963061 

[8.0, 99.0, 84.0, 0.0, 0.0, 35.4, 0.388, 50.0, 0.0] Tamanho 9 

Valor X 8.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 99.0 Media 141.58823529411765 Desvio 30.958569037786383 

Valor X 84.0 Media 70.87058823529412 Desvio 22.047977934787543 

Valor X 0.0 Media 21.323529411764707 Desvio 17.660038445137808 

Valor X 0.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 35.4 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.388 Media 0.5230058823529418 Desvio 0.33234979250870283 

Valor X 50.0 Media 37.33529411764706 Desvio 10.648190177443055 

[8.0, 99.0, 84.0, 0.0, 0.0, 35.4, 0.388, 50.0, 0.0] Tamanho 9 

Valor X 8.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 99.0 Media 109.89244186046511 Desvio 26.72476168863615 

Valor X 84.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 0.0 Media 19.5

Valor X 25.0 Media 21.323529411764707 Desvio 17.660038445137808 

Valor X 70.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 34.0 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.271 Media 0.5230058823529418 Desvio 0.33234979250870283 

Valor X 26.0 Media 37.33529411764706 Desvio 10.648190177443055 

[3.0, 180.0, 64.0, 25.0, 70.0, 34.0, 0.271, 26.0, 0.0] Tamanho 9 

Valor X 3.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 180.0 Media 109.89244186046511 Desvio 26.72476168863615 

Valor X 64.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 25.0 Media 19.584302325581394 Desvio 14.905685536714216 

Valor X 70.0 Media 70.875 Desvio 102.61851038418648 

Valor X 34.0 Media 30.583720930232595 Desvio 7.178764610500314 

Valor X 0.271 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 26.0 Media 31.337209302325583 Desvio 11.800440239963061 

[3.0, 180.0, 64.0, 25.0, 70.0, 34.0, 0.271, 26.0, 0.0] Tamanho 9 

Valor X 3.0 Media 4.847

Valor X 2.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 71.0 Media 109.89244186046511 Desvio 26.72476168863615 

Valor X 70.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 27.0 Media 19.584302325581394 Desvio 14.905685536714216 

Valor X 0.0 Media 70.875 Desvio 102.61851038418648 

Valor X 28.0 Media 30.583720930232595 Desvio 7.178764610500314 

Valor X 0.586 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 22.0 Media 31.337209302325583 Desvio 11.800440239963061 

[2.0, 71.0, 70.0, 27.0, 0.0, 28.0, 0.586, 22.0, 0.0] Tamanho 9 

Valor X 2.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 71.0 Media 141.58823529411765 Desvio 30.958569037786383 

Valor X 70.0 Media 70.87058823529412 Desvio 22.047977934787543 

Valor X 27.0 Media 21.323529411764707 Desvio 17.660038445137808 

Valor X 0.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 28.0 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.586 Media 0.52300588235

...

Valor X 0.0 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.64 Media 0.5230058823529418 Desvio 0.33234979250870283 

Valor X 69.0 Media 37.33529411764706 Desvio 10.648190177443055 

[1.0, 144.0, 82.0, 46.0, 180.0, 46.1, 0.335, 46.0, 1.0] Tamanho 9 

Valor X 1.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 144.0 Media 109.89244186046511 Desvio 26.72476168863615 

Valor X 82.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 46.0 Media 19.584302325581394 Desvio 14.905685536714216 

Valor X 180.0 Media 70.875 Desvio 102.61851038418648 

Valor X 46.1 Media 30.583720930232595 Desvio 7.178764610500314 

Valor X 0.335 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 46.0 Media 31.337209302325583 Desvio 11.800440239963061 

[1.0, 144.0, 82.0, 46.0, 180.0, 46.1, 0.335, 46.0, 1.0] Tamanho 9 

Valor X 1.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 144.0 Media 141.58823529411765 Desvio 30.958569037786383 

Valor X 82.0 Media 70.8

Valor X 60.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 46.0 Media 19.584302325581394 Desvio 14.905685536714216 

Valor X 178.0 Media 70.875 Desvio 102.61851038418648 

Valor X 35.5 Media 30.583720930232595 Desvio 7.178764610500314 

Valor X 0.415 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 24.0 Media 31.337209302325583 Desvio 11.800440239963061 

[1.0, 108.0, 60.0, 46.0, 178.0, 35.5, 0.415, 24.0, 0.0] Tamanho 9 

Valor X 1.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 108.0 Media 141.58823529411765 Desvio 30.958569037786383 

Valor X 60.0 Media 70.87058823529412 Desvio 22.047977934787543 

Valor X 46.0 Media 21.323529411764707 Desvio 17.660038445137808 

Valor X 178.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 35.5 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.415 Media 0.5230058823529418 Desvio 0.33234979250870283 

Valor X 24.0 Media 37.33529411764706 Desvio 10.648190177443055 

[1.0, 108.0, 60.0, 46.

Valor X 45.0 Media 31.337209302325583 Desvio 11.800440239963061 

[1.0, 111.0, 94.0, 0.0, 0.0, 32.8, 0.265, 45.0, 0.0] Tamanho 9 

Valor X 1.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 111.0 Media 141.58823529411765 Desvio 30.958569037786383 

Valor X 94.0 Media 70.87058823529412 Desvio 22.047977934787543 

Valor X 0.0 Media 21.323529411764707 Desvio 17.660038445137808 

Valor X 0.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 32.8 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.265 Media 0.5230058823529418 Desvio 0.33234979250870283 

Valor X 45.0 Media 37.33529411764706 Desvio 10.648190177443055 

[1.0, 111.0, 94.0, 0.0, 0.0, 32.8, 0.265, 45.0, 0.0] Tamanho 9 

Valor X 1.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 111.0 Media 109.89244186046511 Desvio 26.72476168863615 

Valor X 94.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 0.0 Media 19.584302325581394 Desvio 14.905685536714216 

Valor X 0.0 Media 70

Valor X 0.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 23.3 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 0.56 Media 0.5230058823529418 Desvio 0.33234979250870283 

Valor X 53.0 Media 37.33529411764706 Desvio 10.648190177443055 

[2.0, 105.0, 75.0, 0.0, 0.0, 23.3, 0.56, 53.0, 0.0] Tamanho 9 

Valor X 2.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 105.0 Media 109.89244186046511 Desvio 26.72476168863615 

Valor X 75.0 Media 68.63372093023256 Desvio 17.80407206322981 

Valor X 0.0 Media 19.584302325581394 Desvio 14.905685536714216 

Valor X 0.0 Media 70.875 Desvio 102.61851038418648 

Valor X 23.3 Media 30.583720930232595 Desvio 7.178764610500314 

Valor X 0.56 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 53.0 Media 31.337209302325583 Desvio 11.800440239963061 

[2.0, 105.0, 75.0, 0.0, 0.0, 23.3, 0.56, 53.0, 0.0] Tamanho 9 

Valor X 2.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 105.0 Media 141.588235294117

Valor X 57.0 Media 70.875 Desvio 102.61851038418648 

Valor X 46.3 Media 30.583720930232595 Desvio 7.178764610500314 

Valor X 1.096 Media 0.4321598837209308 Desvio 0.3027708011617666 

Valor X 32.0 Media 31.337209302325583 Desvio 11.800440239963061 

[1.0, 81.0, 74.0, 41.0, 57.0, 46.3, 1.096, 32.0, 0.0] Tamanho 9 

Valor X 1.0 Media 4.847058823529411 Desvio 3.7637507490084037 

Valor X 81.0 Media 141.58823529411765 Desvio 30.958569037786383 

Valor X 74.0 Media 70.87058823529412 Desvio 22.047977934787543 

Valor X 41.0 Media 21.323529411764707 Desvio 17.660038445137808 

Valor X 57.0 Media 108.95882352941176 Desvio 154.94375218705324 

Valor X 46.3 Media 34.627647058823534 Desvio 7.411162242985588 

Valor X 1.096 Media 0.5230058823529418 Desvio 0.33234979250870283 

Valor X 32.0 Media 37.33529411764706 Desvio 10.648190177443055 

[8.0, 154.0, 78.0, 32.0, 0.0, 32.4, 0.443, 45.0, 1.0] Tamanho 9 

Valor X 8.0 Media 3.3488372093023258 Desvio 2.9604484801702973 

Valor X 154.0 Media 109.89

In [ ]:
 #probabilities[classValue] = 1
		#for i in range(len(classSummaries)):
		#	mean, stdev = classSummaries[i]
		#	x = inputVector[i]
		#	probabilities[classValue] *= calculateProbability(x, mean, stdev)
	
	#return probabilities